In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
# # Авторизация гугл диска
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd /content/drive/

In [4]:
# создаем датасет для проверки
assert_values = [
    [0, 708930000.0, 688998322.0, 666349089.0, 647380821.0, 630300887.0, 611463914.0, 591781038.0],
    [1, 721780000.0, 700858040.0, 679984420.0, 661238265.0, 644054439.0, 624842853.0, 604260080.0],
    [2, 676197680.0, 657479234.0, 635803739.0, 616560408.0, 599282032.0, 582739733.0, 564951055.0],
    [3, 705906000.0, 684776146.0, 664699620.0, 646675310.0, 629384170.0, 611508787.0, 590382699.0],
    [4, 697650000.0, 677569923.0, 657200106.0, 638066185.0, 620003731.0, 600610165.0, 583305516.0],
    [5, 690886000.0, 670980106.0, 650141712.0, 631199828.0, 613500459.0, 596658378.0, 578937537.0],
    [6, 725780000.0, 703148787.0, 681992058.0, 661459515.0, 641162461.0, 621924929.0, 602339667.0],
]
assert_data = pd.DataFrame(assert_values, columns=['MONTHS_PAST_DEFAULT'] + [f'DEBT_{i}' for i in range(7)])
assert_data = assert_data.set_index('MONTHS_PAST_DEFAULT')

# Загрузка данных

Наша стартовая таблица "Dataset_UP_LGD.csv" со следующими полями:
* ID - идентификационный номер клиента 
* DEBT_0 - сумма долга клиента в месяц дефолта
* PAYMENT_0	- сумма платежа клиента в месяц дефолта
* PAYMENT_1	- сумма платежа клиента в следующем месяце после дефолта клиента
* PAYMENT_2	- сумма платежа клиента через 2 месяца после дефолта клиента	
* PAYMENT_3	- сумма платежа клиента через 3 месяца после дефолта 	
* PAYMENT_4	- сумма платежа клиента через 4 месяца после дефолта 	
* PAYMENT_5	- сумма платежа клиента через 5 месяца после дефолта 	
* PAYMENT_6	- сумма платежа клиента через 6 месяца после дефолта 

In [5]:
data = pd.read_csv('./Dataset_UP_LGD.csv')
data.head()

,ID,DEBT_0,PAYMENT_0,PAYMENT_1,PAYMENT_2,PAYMENT_3,PAYMENT_4,PAYMENT_5,PAYMENT_6
0,1,20000.0,0,0.0,689.0,0.0,0.0,0.0,0.0
1,2,120000.0,0,0.0,1000.0,1000.0,NaN,NaN,NaN
2,3,90000.0,0,1518.0,1500.0,1000.0,1000.0,NaN,NaN
3,4,50000.0,0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,6,50000.0,0,2500.0,1815.0,657.0,1000.0,NaN,NaN


# Task 1. Вычисление долга

Напишите функцию, которая возвращает по каждому клиенту информацию о его долге в каждый месяц после дефолта.

Функция на вход принимает таблицу data

Результатом работы функции является DataFrame с полями:
* ID - идентификационный номер клиента
* MONTHS_PAST_DEFAULT - количество месяцев после дефолта (количество столбцов с долгом != Nan - 1)
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта

In [6]:
def calc_debt(data_):
    payment_cols = [f'PAYMENT_{i}' for i in range(7)]
    notna = data_[payment_cols].notna().sum(axis=1)-1
    data_['MONTHS_PAST_DEFAULT'] = notna

    for i in range(1, 7):
        data_[f'DEBT_{i}'] = data_['DEBT_0'] - sum([data_[f'PAYMENT_{j}'] for j in range(0, i+1)])
    data_ = data_.loc[:, ~data_.columns.str.startswith('PAYMENT')]
    
    return data_

In [7]:
data = calc_debt(data_ = data.copy())

print(data.shape)
assert data.shape == (29034, 9)
assert data.sum().sum() == 19019524231.0
assert data.count().sum() == 174319

(29034, 9)


In [8]:
data.head()

,ID,DEBT_0,MONTHS_PAST_DEFAULT,DEBT_1,DEBT_2,DEBT_3,DEBT_4,DEBT_5,DEBT_6
0,1,20000.0,6,20000.0,19311.0,19311.0,19311.0,19311.0,19311.0
1,2,120000.0,3,120000.0,119000.0,118000.0,NaN,NaN,NaN
2,3,90000.0,4,88482.0,86982.0,85982.0,84982.0,NaN,NaN
3,4,50000.0,6,48000.0,45981.0,44781.0,43681.0,42612.0,41612.0
4,6,50000.0,4,47500.0,45685.0,45028.0,44028.0,NaN,NaN


# Task 2. Группировка наблюдений

Реалиуйте функцию, которая возвращает общую сумму долга для каждой группы наблюдений с одинаковым возрастом дефолта в каждый месяц после дефолта.

Функция на вход принимает таблицу с долгом data

Результатом работы функции является DataFrame с полями:

* MONTHS_PAST_DEFAULT - количество месяцев после дефолта в качестве ИНДЕКСА
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта

Учтите следующее: для тех месяцев, по которым нет данных о долге в таблице должно быть проставлено NaN

In [9]:
def group_debt(data_):
    return data_.groupby('MONTHS_PAST_DEFAULT').sum().drop(columns=['ID']).replace(0, np.nan)

In [10]:
data_gr = group_debt(data_ = data.copy())

assert data_gr.shape == (7, 7)
assert data_gr.sum().sum() == 18584769614.0
assert data_gr.count().sum() == 28

# Task 3. Вычисление коэффициентов

Реализуйте функцию, которая возвращает список коэффициентов переходов для суммы долга между суммами долга в разные месяцы.

Функция на вход принимает сгруппированную таблицу data_gr

Результатом работы функции является np.array с коэффициентами перехода (округлите коэффициенты перехода до 5 знака).

In [11]:
data_gr

,DEBT_0,DEBT_1,DEBT_2,DEBT_3,DEBT_4,DEBT_5,DEBT_6
MONTHS_PAST_DEFAULT,,,,,,,
0,708930000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,721780000.0,700858040.0,NaN,NaN,NaN,NaN,NaN
2,676197680.0,657479234.0,635803739.0,NaN,NaN,NaN,NaN
3,705906000.0,684776146.0,664699620.0,646675310.0,NaN,NaN,NaN
4,697650000.0,677569923.0,657200106.0,638066185.0,620003731.0,NaN,NaN
5,690886000.0,670980106.0,650141712.0,631199828.0,613500459.0,596658378.0,NaN
6,725780000.0,703148787.0,681992058.0,661459515.0,641162461.0,621924929.0,602339667.0


In [12]:
list(range(1))

[0]

In [13]:
def calc_coefs(data_):
    coefs = []
    for i in range(6):
        df = data_.drop(list(range(i+1)))
        df = df[[f'DEBT_{i}', f'DEBT_{i+1}']]
        df = df.sum()
        coefs.append(df[f'DEBT_{i+1}']/df[f'DEBT_{i}'])
        
    return np.round(coefs, 5)

In [14]:
coefs = calc_coefs(data_ = data_gr)
print(coefs)
assert coefs.shape == (6,)
assert round(coefs.sum(), 5) == 5.82191

[0.97075 0.96932 0.97113 0.97096 0.97124 0.96851]


# Task 4. Заполнение пропусков

Реалиуйте функцию, которая заполняет пропуски в сгруппированной таблице долга.

Функция на вход принимает:
* таблицу со сгруппированным долгом data_gr
* список коэффициентов coefs

Результатом работы функции является заполненный DataFrame с полями:

* MONTHS_PAST_DEFAULT - количество месяцев после дефолта в качестве ИНДЕКСА
* DEBT_0 - сумма долга в момент дефолта
* DEBT_1 - сумма долга в следующем месяце после дефолта
* DEBT_2 - сумма долга через 2 есяца после дефолта
* DEBT_3 - сумма долга через 3 есяца после дефолта
* DEBT_4 - сумма долга через 4 есяца после дефолта
* DEBT_5 - сумма долга через 5 есяца после дефолта
* DEBT_6 - сумма долга через 6 есяца после дефолта


In [15]:
data_gr

,DEBT_0,DEBT_1,DEBT_2,DEBT_3,DEBT_4,DEBT_5,DEBT_6
MONTHS_PAST_DEFAULT,,,,,,,
0,708930000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,721780000.0,700858040.0,NaN,NaN,NaN,NaN,NaN
2,676197680.0,657479234.0,635803739.0,NaN,NaN,NaN,NaN
3,705906000.0,684776146.0,664699620.0,646675310.0,NaN,NaN,NaN
4,697650000.0,677569923.0,657200106.0,638066185.0,620003731.0,NaN,NaN
5,690886000.0,670980106.0,650141712.0,631199828.0,613500459.0,596658378.0,NaN
6,725780000.0,703148787.0,681992058.0,661459515.0,641162461.0,621924929.0,602339667.0


In [16]:
def fill_data(data_, coefs):
    for i in range(6):
        data_[f'DEBT_{i+1}'] = data_.apply(lambda x: x[f'DEBT_{i}']*coefs[i] if np.isnan(x[f'DEBT_{i+1}']) else x[f'DEBT_{i+1}'], axis=1 )
    return data_

In [17]:
data_gr_filled = fill_data(data_ = data_gr.copy(),
                           coefs = np.round(coefs, 5)
                           )

assert data_gr_filled.shape == (7, 7)
assert round(data_gr_filled.sum().sum(), 5) == 31578683486.38918
assert data_gr_filled.count().sum() == 49

In [18]:
data_gr_filled

,DEBT_0,DEBT_1,DEBT_2,DEBT_3,DEBT_4,DEBT_5,DEBT_6
MONTHS_PAST_DEFAULT,,,,,,,
0,708930000.0,688193797.5,6.670800e+08,6.478214e+08,6.290087e+08,6.109184e+08,5.916806e+08
1,721780000.0,700858040.0,6.793557e+08,6.597427e+08,6.405838e+08,6.221606e+08,6.025688e+08
2,676197680.0,657479234.0,6.358037e+08,6.174481e+08,5.995174e+08,5.822753e+08,5.639394e+08
3,705906000.0,684776146.0,6.646996e+08,6.466753e+08,6.278959e+08,6.098376e+08,5.906338e+08
4,697650000.0,677569923.0,6.572001e+08,6.380662e+08,6.200037e+08,6.021724e+08,5.832100e+08
5,690886000.0,670980106.0,6.501417e+08,6.311998e+08,6.135005e+08,5.966584e+08,5.778696e+08
6,725780000.0,703148787.0,6.819921e+08,6.614595e+08,6.411625e+08,6.219249e+08,6.023397e+08


Средняя абсолютная ошибка долга по заполненным месяцам

In [19]:
abs((data_gr_filled - assert_data) / assert_data).sum().sum()/((data_gr_filled.shape[0]*data_gr_filled.shape[1] - data_gr_filled.shape[0])/2)

0.0017274558453538337

Теперь можно рассчитать ожидаемый уровень LGD на горизонте 6 месяцев

In [20]:
(data_gr_filled['DEBT_6'] / data_gr_filled['DEBT_0']).mean()

0.8346341869033143